# MAURICIO RODRIGUEZ BERTELLA 100624

# 43. ¿Cuántos usuarios o ips han sido bloqueados al menos una vez y la vez son los revisores de una última versión de un contenido? Calcule la diferencia entre la primera fecha de bloqueo y el promedio de las fechas de revisión correspondientes para cada usuario. (⭐⭐⭐)

In [1]:
import pandas as pd
import numpy as np

In [2]:
logs_cols = ["timestamp","title","action"]
contents_cols = ["revision_timestamp","revisor_username","revisor_ip"]

logs = pd.read_csv("Wikipedia dataset 2c2021/logs.csv", usecols = logs_cols, parse_dates = ["timestamp"])
contents = pd.read_csv("Wikipedia dataset 2c2021/contents.csv", usecols = contents_cols, parse_dates = ["revision_timestamp"])

In [3]:
logs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12188946 entries, 0 to 12188945
Data columns (total 3 columns):
 #   Column     Dtype              
---  ------     -----              
 0   timestamp  datetime64[ns, UTC]
 1   action     object             
 2   title      object             
dtypes: datetime64[ns, UTC](1), object(2)
memory usage: 279.0+ MB


In [4]:
contents.sample()

revision_timestamp revisor_username revisor_ip
2412752 2021-08-07 14:43:14+00:00          Zigurat        NaN

In [5]:
contents["username/ip"] = contents['revisor_username'].fillna(contents['revisor_ip'])
contents = contents.loc[contents["username/ip"].notnull(),["username/ip","revision_timestamp"]]

In [6]:
logs_blocked = logs.loc[(logs.action == "block"),]
logs_blocked["username/ip"] = logs_blocked.title.str.replace("Usuario:","")
logs_blocked

<ipython-input-6-1e701ab60099>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  logs_blocked["username/ip"] = logs_blocked.title.str.replace("Usuario:","")


timestamp action                    title  \
5        2004-12-23 08:28:32+00:00  block    Usuario:212.55.30.157   
287      2004-12-27 17:54:50+00:00  block   Usuario:201.241.228.15   
288      2004-12-27 17:57:12+00:00  block    Usuario:201.255.50.32   
290      2004-12-27 17:59:56+00:00  block     Usuario:84.122.0.158   
313      2004-12-28 01:38:59+00:00  block     Usuario:81.192.12.93   
...                            ...    ...                      ...   
12188865 2021-09-03 02:05:46+00:00  block  Usuario:186.141.200.121   
12188886 2021-09-03 02:24:41+00:00  block        Usuario:Galaxi066   
12188909 2021-09-03 02:52:42+00:00  block      Usuario:Xx.tkpro.xx   
12188924 2021-09-03 03:00:14+00:00  block  Usuario:181.167.104.152   
12188930 2021-09-03 03:05:02+00:00  block    Usuario:190.18.98.138   

              username/ip  
5           212.55.30.157  
287        201.241.228.15  
288         201.255.50.32  
290          84.122.0.158  
313          81.192.12.93  
...                   ...  
12188865  186.141.200.121  
12188886        Galaxi066  
12188909      Xx.tkpro.xx  
12188924  181.167.104.152  
12188930    190.18.98.138  

[342302 rows x 4 columns]

In [7]:
logs_first_block = logs_blocked.groupby("username/ip").agg({"timestamp":"min"}).reset_index()
logs_first_block

username/ip                 timestamp
0       ! ! ! ! ! ! ! ! ! ! ! ! ! ! HAGGEⓇ? 123 2013-03-24 07:12:49+00:00
1         !! !!ПОМОГИТЕ СЕСТРЕ НА ОПЕРАЦИЮ!! !! 2017-11-16 17:22:09+00:00
2                                        !"·$%& 2012-01-04 02:51:53+00:00
3                                !23455666A!wee 2020-05-29 09:34:26+00:00
4                !2o%Uv6PHBj6JfgYmfgqVx!SCEBEP4 2020-05-13 17:40:04+00:00
...                                         ...                       ...
299761                        𝓒𝓸𝓻𝓸𝓷𝓪𝓭𝓸𝓼' 𝓢𝓮𝓻𝓿𝓮𝓻 2021-05-03 00:36:55+00:00
299762                               𝕊𝟜𝕒𝕟𝕥𝕚𝟚𝟘𝟘𝟟 2021-03-19 22:44:47+00:00
299763                        𝙽𝚘𝙹𝚘𝚍𝚊𝚜𝙿𝚒𝚗𝚜𝚑𝚒𝙿𝚞𝚝𝚡 2021-06-03 17:42:27+00:00
299764                                        😂 2021-07-11 10:51:18+00:00
299765                                        🤖 2021-07-11 10:51:29+00:00

[299766 rows x 2 columns]

In [8]:
logs_merge = logs_first_block.merge(contents, on="username/ip",how="inner")
logs_merge.columns = ["username/ip","first_block_timestamp","revision_timestamps_mean"]
logs_merge

username/ip     first_block_timestamp  revision_timestamps_mean
0         "BF CLUB" 2019-09-22 21:16:39+00:00 2019-09-19 23:56:53+00:00
1         "BF CLUB" 2019-09-22 21:16:39+00:00 2019-09-10 08:11:44+00:00
2         "BF CLUB" 2019-09-22 21:16:39+00:00 2019-10-06 21:23:46+00:00
3         "BF CLUB" 2019-09-22 21:16:39+00:00 2019-10-06 17:40:52+00:00
4         "BF CLUB" 2019-09-22 21:16:39+00:00 2019-10-01 18:19:09+00:00
...             ...                       ...                       ...
1639160        東急電鉄 2010-10-16 07:34:20+00:00 2010-10-15 10:07:21+00:00
1639161        爆笑連合 2010-08-08 20:16:15+00:00 2010-07-23 00:39:47+00:00
1639162        貨車太郎 2010-08-08 20:13:37+00:00 2010-08-07 21:45:16+00:00
1639163        貨車太郎 2010-08-08 20:13:37+00:00 2010-08-07 04:41:44+00:00
1639164        鳩山邦夫 2010-08-08 19:59:46+00:00 2010-07-29 01:40:08+00:00

[1639165 rows x 3 columns]

In [9]:
logs_merge["revision_timestamps_mean"] = pd.to_datetime(logs_merge.revision_timestamps_mean, utc=True).values.astype(np.int64)
logs_merge = pd.DataFrame(pd.to_datetime(logs_merge.groupby(\
            ['username/ip','first_block_timestamp'])["revision_timestamps_mean"].mean(), utc=True)).reset_index()
logs_merge

username/ip     first_block_timestamp            revision_timestamps_mean
0      "BF CLUB" 2019-09-22 21:16:39+00:00 2019-09-30 08:39:28.714285568+00:00
1      $leazytor 2012-11-30 19:33:11+00:00           2012-11-26 03:44:36+00:00
2     &beer&love 2019-08-30 15:41:10+00:00 2018-09-23 16:08:15.466694656+00:00
3          -dam- 2014-04-06 15:07:31+00:00 2013-02-06 16:01:48.750000128+00:00
4          -seb- 2012-02-18 05:22:29+00:00 2011-12-18 14:26:40.545454592+00:00
...          ...                       ...                                 ...
5855         日本語 2009-12-16 17:14:35+00:00    2009-10-22 03:19:22.500000+00:00
5856        東急電鉄 2010-10-16 07:34:20+00:00           2010-09-10 18:20:35+00:00
5857        爆笑連合 2010-08-08 20:16:15+00:00           2010-07-23 00:39:47+00:00
5858        貨車太郎 2010-08-08 20:13:37+00:00           2010-08-07 13:13:30+00:00
5859        鳩山邦夫 2010-08-08 19:59:46+00:00           2010-07-29 01:40:08+00:00

[5860 rows x 3 columns]

## ¿Cuántos usuarios o ips han sido bloqueados al menos una vez y la vez son los revisores de una última versión de un contenido?

In [10]:
len(logs_merge)

5860

## Calcule la diferencia entre la primera fecha de bloqueo y el promedio de las fechas de revisión correspondientes para cada usuario

In [11]:
logs_merge["diff"] = (logs_merge.revision_timestamps_mean-logs_merge.first_block_timestamp)
logs_merge[["username/ip","diff"]]

username/ip                          diff
0      "BF CLUB"     7 days 11:22:49.714285568
1      $leazytor             -5 days +08:11:25
2     &beer&love -341 days +00:27:05.466694656
3          -dam- -424 days +00:54:17.750000128
4          -seb-  -62 days +09:04:11.545454592
...          ...                           ...
5855         日本語     -56 days +10:04:47.500000
5856        東急電鉄            -36 days +10:46:15
5857        爆笑連合            -17 days +04:23:32
5858        貨車太郎             -2 days +16:59:53
5859        鳩山邦夫            -11 days +05:40:22

[5860 rows x 2 columns]